In [1]:
# to make the .py script runnable
#!/usr/bin/env python

In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

In [1]:
import os

# 2.3 Importing data and import/read from and export/ dump to SQL (done)

## 2.3.1 CSV Files

Reading a CSV is as simple as calling the read_csv function. By default, the `read_csv` function expects the column separator to be a comma, but you can change that using the sep parameter.

Syntax: `pd.read_csv(filepath, sep=, header=, names=, skiprows=, na_values= ... )`

The example used in this section is a dataframe, an object inherent to pandas. We will take an in depth look at dataframes in the next section.

### 2.3.1.1 Inspect file without importing it

In [4]:
!wc -l data/titanic.csv

     892 data/titanic.csv


In [5]:
!head -5 data/titanic.csv #note the difference between the first line and the other lines

In [6]:
!tail -5 data/titanic.csv

In [7]:
lines = !head -5 data/titanic.csv #note the difference between the first line and the other lines

In [8]:
print(lines)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked', '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S', '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C', '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S', '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S']


### 2.3.1.2 Importing a CSV file

In [9]:
df_titanic = pd.read_csv('data/titanic.csv')

In [10]:
type(df_titanic)

pandas.core.frame.DataFrame

In [11]:
df_titanic.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [12]:
df_titanic.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


## 2.3.2 Support for SQL Databases

Pandas also has some support for reading/writing DataFrames directly from/to a database. 

You'll typically just need to pass a connection object to the `read_frame` or `write_frame` functions within the pandas.io module.

***Note*** `write_frame` executes as a series of `INSERT INTO` statements and thus trades speed for simplicity. If you're writing a large DataFrame to a database, it might be quicker to write the DataFrame to CSV and load that directly using the database's file import arguments.

In [13]:
os.getcwd() + '/data'

'/Users/AVNH/0 - Crunch Analytics/Notebooks/data'

In [14]:
os.listdir(os.getcwd() + '/data')

['store_info.csv',
 'towed.db',
 'all_the_misses.csv',
 'titanic.csv',
 'features.csv',
 'historic_sales.csv']

In [15]:
dblist = [x for x in os.listdir(os.getcwd() + '/data') if '.db' in x]

for db in dblist:
    print(db)

towed.db


In [ ]:
help(pd.read_sql) #Delete the '#' to see some extra info

In [16]:
from pandas.io import sql
import sqlite3 

conn = sqlite3.connect('data/towed.db')
query = "SELECT * FROM towed WHERE make = 'FORD';"

results = pd.read_sql(query, con=conn)
print(results.head())

         date  make style model color    plate state             towed_to  \
0  11/18/2014  FORD    LL         BLK  S105053    IL        10300 S. Doty   
1  11/18/2014  FORD    LL         WHI  N104269    IL  400 E. Lower Wacker   
2  11/18/2014  FORD    2D         WHI  V519776    IL    701 N. Sacramento   
3  11/18/2014  FORD    PK    TK   RED             IL    701 N. Sacramento   
4  11/18/2014  FORD    4D         GRY             IL    701 N. Sacramento   

    facility_phone  inventory_num  
0  (773) 568-8495         2750424  
1   (312) 744-7550         917123  
2   (773) 265-1846        1532925  
3   (773) 265-7605        6798362  
4   (773) 265-7605        6798317  


In [17]:
pd.read_sql("SELECT distinct make, count(*) from towed group by 1 order by 2 desc limit 10", conn)

,make,count(*)
0,CHEV,870
1,FORD,605
2,DODG,386
3,PONT,368
4,TOYT,292
5,BUIC,283
6,NISS,273
7,HOND,251
8,CHRY,195
9,OLDS,144


In [19]:
from pandas.io import sql
import sqlite3 

conn = sqlite3.connect('data/towed.db')

#use to_sql instead of write_frame
df_titanic.to_sql("tbTitanic", con=conn)

In [20]:
from pandas.io import sql
import sqlite3 

conn = sqlite3.connect('data/towed.db')
#query = "select * from sqlite_master where type = 'table';"
query = "SELECT * FROM tbTitanic;"

df_titanic2 = pd.read_sql(query, con=conn)
print(df_titanic2.head())

   index  PassengerId  Survived  Pclass  \
0      0            1         0       3   
1      1            2         1       1   
2      2            3         1       3   
3      3            4         1       1   
4      4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500  None        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250  None        S  
3      0            113803  53.1000  C123        S  
4      0     

In [21]:
#dump one table

import sqlite3
import pandas as pd

db = sqlite3.connect('data/towed.db')
table = pd.read_sql_query("SELECT * from towed", db)
#table.to_csv(table_name + '.csv', index_label='index')
print(table)

            date  make style model color     plate state             towed_to  \
0     11/18/2014  FORD    LL         BLK   S105053    IL        10300 S. Doty   
1     11/18/2014  HOND    4D   ACC   BLK   S415270    IL  400 E. Lower Wacker   
2     11/18/2014  CHRY    VN         SIL   V847641    IL    701 N. Sacramento   
3     11/18/2014  HYUN    4D         SIL   N756530    IL  400 E. Lower Wacker   
4     11/18/2014  TOYT    4D         WHI   K702211    IL  400 E. Lower Wacker   
5     11/18/2014  PONT    VN    TK   TAN   V415912    IL    701 N. Sacramento   
6     11/18/2014  JEEP    LL    TK   RED   717R342    IL    701 N. Sacramento   
7     11/18/2014  CHEV    4D         BLK   E255028    IL  400 E. Lower Wacker   
8     11/18/2014  SUBA    4D         SIL   FOA2720    OH  400 E. Lower Wacker   
9     11/18/2014  VOLK    2D         BLK   R775959    IL  400 E. Lower Wacker   
10    11/18/2014  MERZ    4D         BLK   S796032    IL        10300 S. Doty   
11    11/18/2014  BUIC    4D

In [24]:
#dump all tables

import sqlite3
import pandas as pd

def to_csv():
    db = sqlite3.connect('data/towed.db')
    cursor = db.cursor()
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    cursor.execute(query)
    tables = cursor.fetchall()
    print(tables)
    for table_name in tables:
        print(table_name)
        #table_name = table_name[0]
        sql = "SELECT * from %s LIMIT 3;" % table_name
        print(sql)
        table = pd.read_sql_query(sql, db)
        #table.to_csv(table_name + '.csv', index_label='index')
        print (table.head())
        
to_csv()

[('towed',), ('dbTitanic',), ('tbTitanic',)]
('towed',)
SELECT * from towed LIMIT 3;
         date  make style model color    plate state             towed_to  \
0  11/18/2014  FORD    LL         BLK  S105053    IL        10300 S. Doty   
1  11/18/2014  HOND    4D   ACC   BLK  S415270    IL  400 E. Lower Wacker   
2  11/18/2014  CHRY    VN         SIL  V847641    IL    701 N. Sacramento   

    facility_phone  inventory_num  
0  (773) 568-8495         2750424  
1   (312) 744-7550         917129  
2   (773) 265-7605        6798366  
('dbTitanic',)
SELECT * from dbTitanic LIMIT 3;
   index  PassengerId  Survived  Pclass  \
0      0            1         0       3   
1      1            2         1       1   
2      2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2         

---

Read: 

1. Homepage http://www.sqlalchemy.org/
2. Engines http://docs.sqlalchemy.org/en/latest/core/engines.html

---

## 2.3.3 Reading from the Clipboard!

This is as straight forward as it ought to be.

Example: `df_2 = pd.read_clipboard(); df_2.head()`

---